In [70]:
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.objectives import *
from lasagne.updates import *
import numpy as np
import theano
from theano import tensor as T

In [19]:
import cPickle as pickle
import gzip

In [25]:
from sklearn.preprocessing import OneHotEncoder

In [62]:
def lstm(num_units=10):
    seqlen = 28*28
    l_in = InputLayer((None, seqlen, 256))
    l_lstm = LSTMLayer(l_in, num_units=num_units)
    l_shp = ReshapeLayer(l_lstm, (-1, num_units))
    l_dense = DenseLayer(l_shp, num_units=256, nonlinearity=softmax)
    l_shp2 = ReshapeLayer(l_dense, (-1, seqlen, 256))
    return l_shp2

In [94]:
l_out = lstm()
for layer in get_all_layers(l_out):
    print layer, layer.output_shape

<lasagne.layers.input.InputLayer object at 0x128c6b810> (None, 784, 256)
<lasagne.layers.recurrent.LSTMLayer object at 0x128c6b8d0> (None, 784, 10)
<lasagne.layers.shape.ReshapeLayer object at 0x1296eabd0> (None, 10)
<lasagne.layers.dense.DenseLayer object at 0x128c7f290> (None, 256)
<lasagne.layers.shape.ReshapeLayer object at 0x128c672d0> (None, 784, 256)


In [95]:
X = T.tensor3('X')
net_out = get_output(l_out, X)

In [96]:
eps=1e-1
loss_pre = categorical_crossentropy(net_out[:,0:-1,:], X[:,1::,:])
loss = loss_pre.mean()
params = get_all_params(l_out, trainable=True)
updates = rmsprop(loss, params, learning_rate=0.01)
train_fn = theano.function([X], loss, updates=updates)
loss_fn = theano.function([X], loss_pre)

In [105]:
net_out.eval({X:X_train_hot[0:1]}).shape

(1, 784, 256)

-----

In [33]:
with gzip.open("../../../data/mnist.pkl.gz") as f:
    dat = pickle.load(f)

In [34]:
train_data, _, _ = dat

In [35]:
X_train, _ = train_data
X_train = (X_train*256).astype("uint8")
np.min(X_train), np.max(X_train)

(0, 255)

In [50]:
def mnist_to_one_hot(X_train):
    total = []
    for b in range(0, X_train.shape[0]):
        seqs = []
        for i in range(0, X_train[b].shape[0]):
            one_hot = np.zeros((256)).astype("float32")
            one_hot[ X_train[b][i] ] = 1.0
            seqs.append(one_hot)
        seqs = np.asarray(seqs, dtype="float32")
        total.append(seqs)
    total = np.asarray(total, dtype="float32")
    return total

In [52]:
X_train_hot = mnist_to_one_hot(X_train[0:100])
X_train_hot.shape

(100, 784, 256)

In [102]:
for epoch in range(0,100):
    print train_fn(X_train_hot)

5.36227188242
5.28196104307
5.16609311985
4.97353885543
4.62695954068
4.04176820795
3.54415922595
3.1761333631
2.76657923779
2.49782194477
2.29488627229
2.12527501744
1.98474474864
1.86991850543
1.77596041071
1.6992545978
1.63678179313
1.58591554061
1.54440373341
1.50580250194
1.478849668
1.4573801078
1.43967061109
1.42500537481
1.41282132703
1.40266933373
1.39418839118
1.38708661094
1.38112674812
1.37611503459
1.37189249462
1.36832815074
1.36531367969
1.36275918628
1.36058984341
1.35874320569
1.35716704969
1.35581762771
1.35465824793


KeyboardInterrupt: 

In [118]:
# (bs=1, 2, 10)
tmp1 = np.asarray([[[0.2,0.2,0.4,0.2],[0.7,0.1,0.1,0.1]]])
print tmp1
print tmp1.shape

[[[ 0.2  0.2  0.4  0.2]
  [ 0.7  0.1  0.1  0.1]]]
(1, 2, 4)


In [119]:
mask= np.tile( np.asarray([0,1,2,3]), (1, 2, 1) )
print mask
print mask.shape

[[[0 1 2 3]
  [0 1 2 3]]]
(1, 2, 4)


In [125]:
(tmp1*mask)

array([[[ 0. ,  0.2,  0.8,  0.6],
        [ 0. ,  0.1,  0.2,  0.3]]])

In [127]:
print (tmp1*mask).sum(axis=2)
print (tmp1*mask).sum(axis=2).shape

[[ 1.6  0.6]]
(1, 2)
